# NetGuard APK Builder (Google Colab)

This notebook allows you to compile your KivyMD project into an Android APK using Google's cloud servers. This is the easiest way to build your app if you don't have a Linux/WSL environment set up on your local machine.

### Step 1: Install System Dependencies
Run this cell to install the necessary Android and Python build tools.

In [ ]:
!sudo apt update
!sudo apt install -y git zip unzip openjdk-17-jdk python3-pip autoconf libtool pkg-config zlib1g-dev libncurses5-dev libncursesw5-dev libtinfo5 cmake libffi-dev libssl-dev
!pip install --upgrade buildozer cython virtualenv

### Step 2: Upload Your Project
1. In the left sidebar of Colab, click the **Folder icon**.
2. Right-click in the file list and select **Upload**.
3. Upload a ZIP file of your project (e.g., `NetGuard_Proj.zip`) or individual files (`main.py`, `buildozer.spec`, etc.).

**Alternatively**, run the cell below to create a simple upload prompt:

In [ ]:
from google.colab import files
import os
import shutil

uploaded = files.upload()

# Auto-Unzip if a zip file was uploaded
for fn in uploaded.keys():
    if fn.endswith('.zip'):
        print(f"Unzipping {fn}...")
        !unzip -o "{fn}"

### Step 2.5: Fix File Structure (Important!)
Sometimes unzipping creates a subfolder (e.g., `NetGuard proj/`). Buildozer needs `buildozer.spec` to be in the **current folder**.
Run this cell to find your project files and move them to the root if needed.

In [ ]:
import os
import shutil

# Look for buildozer.spec
target_file = 'buildozer.spec'
found_dir = None

if os.path.exists(target_file):
    print("✅ buildozer.spec found in root. Good to go!")
else:
    # Search subdirectories
    print("🔍 Searching for buildozer.spec...")
    for root, dirs, files in os.walk('.'):
        if target_file in files:
            found_dir = root
            print(f"Found project in: {found_dir}")
            break

    if found_dir:
        print("🚀 Moving files to root directory...")
        for item in os.listdir(found_dir):
            s = os.path.join(found_dir, item)
            d = os.path.join('.', item)
            if os.path.exists(d):
                if os.path.isdir(d):
                    shutil.rmtree(d)
                else:
                    os.remove(d)
            shutil.move(s, d)
        print("✅ Files moved. Ready to build.")
    else:
        print("❌ ERROR: buildozer.spec not found inside the uploaded files. Please check your zip file.")

### Step 3: Start the Build
Run the following command to start the build. 

**Note:** First time building will take 15-20 minutes as it downloads the Android SDK/NDK.

In [ ]:
import os
if not os.path.exists("buildozer.spec"):
    print("STOP! buildozer.spec is missing. Did you upload the files?")
else:
    !buildozer android debug

### Step 4: Download Your APK
Once the build finishes, your APK will be in the `bin/` folder. Run this cell to download it to your computer.

In [ ]:
import os
apk_list = [f for f in os.listdir('bin') if f.endswith('.apk')]
if apk_list:
    apk_file = apk_list[0]
    print(f"Downloading {apk_file}...")
    files.download(os.path.join('bin', apk_file))
else:
    print("No APK found. Did the build succeed?")